In [1]:
from pulp import *
import numpy as np
# initialize the model
prob = LpProblem("warehouseTransportation", LpMinimize)

# Creates a dictionary for the number of units of supply for each supply node
supply = {0: 1000,
        1: 4000}

# Creates a dictionary for the number of units of demand for each demand node
demand = {0: 500,
        1: 900,
        2: 1800,
        3: 200,
        4: 700}

T = len(demand)
W = len(supply)
# ---------------------
# VARIABLES
# ---------------------
# Creates a list of all demand nodes
Warehouses = [0,1]
# Creates a list of all the supply nodes
Bars = [0, 1, 2, 3, 4]

# Creates a list of costs of each transportation path
costs = [   #Bars
        #1 2 3 4 5
        [2,4,5,2,1],#A  Warehouses
        [3,1,3,2,3] #B
        ]
# Creates a list of tuples containing all the possible routes for transport
Routes = [(w,b) for w in Warehouses for b in Bars]
# A dictionary called route_vars is created to contain the referenced variables (the routes)
route_vars = LpVariable.dicts("Route",(Warehouses,Bars),0,None,LpInteger)


# The objective function is added to prob first
prob += lpSum([route_vars[w][b]*costs[w][b] for (w,b) in Routes]), "Sum of Transporting Costs"
    
    

# The supply maximum constraints are added to prob for each supply node (warehouse)
for b in Bars:
    prob += sum([route_vars[w][b] for w in Warehouses]) >= demand[b]

# The demand minimum constraints are added to prob for each demand node (bar)
for w in Warehouses:
    prob += lpSum([route_vars[w][b] for b in Bars]) <= supply[w]


prob.writeLP("warehouseTransportation.lp")
    
status = prob.solve(GLPK(options=["--ranges","warehouseTransportation.sen"]))
print(status)

# Each of the variables is printed with it’s resolved optimum value
for v in prob.variables():
    print (v.name, "=", v.varValue)

# The optimised objective function value is printed to the screen
print ("Total Cost of Transportation = ", prob.objective.value())

1
Route_0_0 = 300
Route_0_1 = 0
Route_0_2 = 0
Route_0_3 = 0
Route_0_4 = 700
Route_1_0 = 200
Route_1_1 = 900
Route_1_2 = 1800
Route_1_3 = 200
Route_1_4 = 0
Total Cost of Transportation =  8600


In [2]:
#%load aavinMilk.sen


# Solve using Gurobi

In [3]:
from gurobipy import *

# Warehouse demand in thousands of units
demand = [15, 18, 14, 20]

# Plant capacity in thousands of units
capacity = [20, 22, 17, 19, 18]

# Fixed costs for each plant
fixedCosts = [12000, 15000, 17000, 13000, 16000]

# Transportation costs per thousand units
transCosts = [[4000, 2000, 3000, 2500, 4500],
              [2500, 2600, 3400, 3000, 4000],
              [1200, 1800, 2600, 4100, 3000],
              [2200, 2600, 3100, 3700, 3200]]

# Range of plants and warehouses
plants = range(len(capacity))
warehouses = range(len(demand))

# Model
m = Model("facility")

# Plant open decision variables: open[p] == 1 if plant p is open.
open = m.addVars(plants,
                 vtype=GRB.BINARY,
                 obj=fixedCosts,
                 name="open")

# Transportation decision variables: transport[w,p] captures the
# optimal quantity to transport to warehouse w from plant p
transport = m.addVars(warehouses, plants, obj=transCosts, name="trans")

# You could use Python looping constructs and m.addVar() to create
# these decision variables instead.  The following would be equivalent
# to the preceding two statements...
#
#open = []
#for p in plants:
#  open.append(m.addVar(vtype=GRB.BINARY,
#                       obj=fixedCosts[p],
#                       name="open[%d]" % p))
#
#transport = []
#for w in warehouses:
#  transport.append([])
#  for p in plants:
#    transport[w].append(m.addVar(obj=transCosts[w][p],
#                                 name="trans[%d,%d]" % (w, p)))

# The objective is to minimize the total fixed and variable costs
m.modelSense = GRB.MINIMIZE

# Production constraints
# Note that the right-hand limit sets the production to zero if the plant
# is closed
m.addConstrs(
    (transport.sum('*',p) <= capacity[p]*open[p] for p in plants),
    "Capacity")

# Using Python looping constructs, the preceding would be...
#
#for p in plants:
#  m.addConstr(sum(transport[w][p] for w in warehouses) <= capacity[p] * open[p],
#              "Capacity[%d]" % p)

# Demand constraints
m.addConstrs(
    (transport.sum(w) == demand[w] for w in warehouses),
    "Demand")

# ... and the preceding would be ...
#for w in warehouses:
#  m.addConstr(sum(transport[w][p] for p in plants) == demand[w], "Demand[%d]" % w)

# Save model
m.write('facilityPY.lp')

# Guess at the starting point: close the plant with the highest fixed costs;
# open all others

# First, open all plants
for p in plants:
    open[p].start = 1.0

# Now close the plant with the highest fixed cost
print('Initial guess:')
maxFixed = max(fixedCosts)
for p in plants:
    if fixedCosts[p] == maxFixed:
        open[p].start = 0.0
        print('Closing plant %s' % p)
        break
print('')

# Use barrier to solve root relaxation
m.Params.method = 2

# Solve
m.optimize()

# Print solution
print('\nTOTAL COSTS: %g' % m.objVal)
print('SOLUTION:')
for p in plants:
    if open[p].x > 0.99:
        print('Plant %s open' % p)
        for w in warehouses:
            if transport[w,p].x > 0:
                print('  Transport %g units to warehouse %s' % \
                      (transport[w,p].x, w))
    else:
        print('Plant %s closed!' % p)

Academic license - for non-commercial use only
Initial guess:
Closing plant 2

Changed value of parameter method to 2
   Prev: -1  Min: -1  Max: 5  Default: -1
Optimize a model with 9 rows, 25 columns and 45 nonzeros
Variable types: 20 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+03, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 2e+01]

Loaded MIP start with objective 210500

Presolve time: 0.00s
Presolved: 9 rows, 25 columns, 45 nonzeros
Variable types: 20 continuous, 5 integer (5 binary)
Presolved: 9 rows, 25 columns, 45 nonzeros

Root barrier log...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.000e+01
 Factor NZ  : 4.500e+01
 Factor Ops : 2.850e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   7.94290841e+05 -2.24842916e+05  7.25e+00 3.75e+03 